In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [6]:
import ompy as om
import numpy as np
import pymc3 as pm
import arviz as az
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import logging
from scipy.interpolate import interp1d
from model_op import calculate_FG, LogLike, LogLike2, FG_loglike, loglike

In [7]:
raw = om.example_raw('Dy164')

raw.cut_diagonal(E1=(800, 0), E2=(7500, 7300))
raw.cut('Ex', 0, 8400)

raw.values = np.around(raw.values)
raw.remove_negative()
raw.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.QuadMesh at 0x139e10d00>,
 <AxesSubplot:xlabel='$\\gamma$-ray energy $E_{\\gamma}$', ylabel='Excitation energy $E_{x}$'>,
 <Figure size 640x480 with 2 Axes>)

In [8]:
folderpath = "../OCL_response_functions/nai2012_for_opt13"
Eg = raw.Eg

# Experimental relative FWHM at 1.33 MeV of resulting array
fwhm_abs = 90.44 # (90/1330 = 6.8%)

# Magne recommends 1/10 of the actual resolution for unfolding purposes
response = om.Response(folderpath)
R_ompy_unf, R_tab_unf = response.interpolate(Eg, fwhm_abs=fwhm_abs/10, return_table=True)
R_ompy_view, _ = response.interpolate(Eg, fwhm_abs=fwhm_abs, return_table=True)
fthreshold = interp1d([30., 80., 122., 183., 244., 294., 344., 562., 779., 1000.],
                      [0.0, 0.0, 0.0, 0.06, 0.44, 0.60, 0.87, 0.99, 1.00, 1.00],
                      fill_value="extrapolate")

def apply_detector_threshold(response, table, fthreshold):
    thres = fthreshold(response.Eg)
    response.values = response.values * thres
    # renormalize
    response.values = om.div0(response.values, response.values.sum(axis=1)[:, np.newaxis])
    table["eff_tot"] *= thres
apply_detector_threshold(R_ompy_unf, R_tab_unf, fthreshold)
R_ompy_view.plot(scale='log', vmin=1e-4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.QuadMesh at 0x138298310>,
 <AxesSubplot:xlabel='$\\gamma$-ray energy $E_{\\gamma}$', ylabel='Excitation energy $E_{x}$'>,
 <Figure size 640x480 with 2 Axes>)

In [9]:
# With compton subtraction and all tweaks
unfolder= om.Unfolder(response=R_ompy_unf)
unfolder.response_tab = R_tab_unf
# Magne suggests some "tweaks" for a better unfolding performance. Default is 1 for all.
unfolder.FWHM_tweak_multiplier = {"fe": 1., "se": 1.1,
                                     "de": 1.3, "511": 0.9}
firstgen = om.FirstGeneration()

In [10]:
logger = om.introspection.get_logger('ensemble', 'INFO')

# Tell the `Ensemble` class which raw spectrum, what kind of undolfer and first
# generations method to use.
# Note: This will have the same setting as above. We could for example have
# set the first generations method to use a different "valley_collection", or a
# differnt type of "multiplicity_estimation"
ensemble = om.Ensemble(raw=raw)
ensemble.unfolder = unfolder
ensemble.first_generation_method = firstgen
# Generates N perturbated members;
# the `regernerate` flag ensures, that we don't load from disk; which might result in expected results
# if we have changed something in the input `raw` matrix.
ensemble.generate(25, regenerate=True)
E_rebinned = np.arange(100., 8500, 200)
ensemble.rebin(E_rebinned, member="firstgen")

2021-11-12 14:48:18,745 - ompy.ensemble - INFO - Start normalization with 7 cpus
2021-11-12 14:48:18,819 - ompy.ensemble - INFO - Generating/loading 0
2021-11-12 14:48:18,851 - ompy.ensemble - INFO - Generating/loading 1


  0%|          | 0/25 [00:00<?, ?it/s]

2021-11-12 14:48:18,889 - ompy.ensemble - INFO - Generating/loading 2
2021-11-12 14:48:18,925 - ompy.ensemble - INFO - Generating/loading 3
2021-11-12 14:48:18,961 - ompy.ensemble - INFO - Generating/loading 4
2021-11-12 14:48:18,999 - ompy.ensemble - INFO - Generating/loading 5
2021-11-12 14:48:19,036 - ompy.ensemble - INFO - Generating/loading 6
2021-11-12 14:48:23,031 - ompy.ensemble - INFO - Generating/loading 7
2021-11-12 14:48:23,093 - ompy.ensemble - INFO - Generating/loading 8
2021-11-12 14:48:23,147 - ompy.ensemble - INFO - Generating/loading 9
2021-11-12 14:48:23,259 - ompy.ensemble - INFO - Generating/loading 10
2021-11-12 14:48:23,299 - ompy.ensemble - INFO - Generating/loading 11
2021-11-12 14:48:23,341 - ompy.ensemble - INFO - Generating/loading 12
2021-11-12 14:48:23,378 - ompy.ensemble - INFO - Generating/loading 13
2021-11-12 14:48:28,157 - ompy.ensemble - INFO - Generating/loading 14
2021-11-12 14:48:28,315 - ompy.ensemble - INFO - Generating/loading 15
2021-11-12 14:

  0%|          | 0/25 [00:00<?, ?it/s]

In [11]:
firstgen = ensemble.get_firstgen(0)
firstgen = om.Matrix(Ex=firstgen.Ex, Eg=firstgen.Eg, values=np.mean([mat.values for mat in ensemble.get_firstgen(range(25))], axis=0))
firstgen_std = om.Matrix(Ex=firstgen.Ex, Eg=firstgen.Eg, values=np.std([mat.values for mat in ensemble.get_firstgen(range(25))], axis=0))
firstgen.trapezoid(4400, 7700, 1300, 7900, inplace=True)
firstgen_std.trapezoid(4400, 7700, 1300, 7900, inplace=True)

In [12]:
firstgen.plot()
firstgen_std.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.QuadMesh at 0x1382ae970>,
 <AxesSubplot:xlabel='$\\gamma$-ray energy $E_{\\gamma}$', ylabel='Excitation energy $E_{x}$'>,
 <Figure size 640x480 with 2 Axes>)

In [13]:
# We run the extractor on the ensemble just to have a reference

In [14]:
trapezoid_cut = om.Action('matrix')
trapezoid_cut.trapezoid(Ex_min=4400, Ex_max=7700, Eg_min=1300, Eg_max=7900, inplace=True)
extractor = om.Extractor()
extractor.trapezoid = trapezoid_cut
extractor.extract_from(ensemble)

  0%|          | 0/25 [00:00<?, ?it/s]

nld #0 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #1 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #2 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #3 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #4 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #5 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #6 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #7 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #8 contains nan's.
Consider removing them e.g. with:
# for nld in extractor.nld:
#     nld.cut_nan()

nld #9 contains nan's.
Consider removing them 

In [15]:
dE = firstgen.Ex[1] - firstgen.Ex[0]
print(dE)
print(firstgen.Ex[0] - firstgen.Eg[0])
print((firstgen.Ex[0] - firstgen.Eg[0])/dE)

k0 = int((firstgen.Ex[0] - firstgen.Eg[0])/dE)

Ef = []
for ex in firstgen.Ex:
    for eg in firstgen.Eg:
        Ef.append(ex-eg)
# We only keep those that are larger than one
Ef = np.array(Ef)
Ef = Ef[Ef >= 0]
# Keep only unique
Ef = np.unique(Ef)

# Next we can get the index for NLDs
index_NLD = []
for i, ex in enumerate(firstgen.Ex):
    idx = []
    for j, eg in enumerate(firstgen.Eg):
        if ex - eg < 0:
            idx.append(len(Ef))
            continue
        idx.append(i-j+k0)
    index_NLD.append(idx)
index_NLD = np.array(index_NLD)

# Lastly we can make the index for gSFs
index_GSF = []
for i, ex in enumerate(firstgen.Ex):
    idx = []
    for j, eg in enumerate(firstgen.Eg):
        if eg > ex:
            idx.append(len(firstgen.Eg)-1)
            continue
        idx.append(j)
    index_GSF.append(idx)
index_GSF = np.array(index_GSF)
    
index_keep = []
for i, ex in enumerate(firstgen.Ex):
    for j, eg in enumerate(firstgen.Eg):
        if eg > ex:
            continue
        index_keep.append(i*len(firstgen.Eg) + j)
index_keep = np.array(index_keep)


print(index_NLD.shape)
print(index_GSF.shape)
print(index_keep)

200.0
3200.0
16.0
(17, 34)
(17, 34)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 204 205 206 207 208 209 210 211 212
 213 214 215 216 217 218 219 220 221 222 223 224 225 226 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 260 261 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287
 288 289 290 291 292 293 294 295 296 306 307 308 309 310 311 312 313 314
 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 340
 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358
 359 360 361 36

In [17]:
nld0 = extractor.nld[0].copy()
gsf0 = extractor.gsf[0].copy()

nld0_no_nan = nld0.copy()
nld0_no_nan.cut_nan()
nld0_no_nan.to_MeV()


gsf0_no_nan = gsf0.copy()
gsf0_no_nan.cut_nan()
gsf0_no_nan.to_MeV()

E_nld = nld0_no_nan.E.copy()
print(E_nld)

Sn = 7.658 # MeV
rhoSn = [1.916E+06, 3.503E+05]
Γ_γ_obs = [113., 13.]
nld_range_low = [0.6, 1.7]
nld_range_high = [3., 6.5]

N_ρ = len(nld0_no_nan)
N_T = len(firstgen.Eg)-1
print(len(gsf0_no_nan))

# bins that we compare with the discrete
idx_discrete = np.array(range(nld0_no_nan.index(nld_range_low[0]), nld0_no_nan.index(nld_range_low[1])), dtype=int)
idx_model = np.array(range(nld0_no_nan.index(nld_range_high[0]), nld0_no_nan.index(nld_range_high[1])), dtype=int)
idx_pre = np.array(range(idx_discrete[0]), dtype=int)
idx_mid = np.array(range(idx_discrete[-1]+1, idx_model[0]), dtype=int)
idx_end = np.array(range(idx_model[-1]+1, N_ρ), dtype=int)

print(idx_pre)
print(idx_discrete)
print(idx_mid)
print(idx_model)
print(idx_end)

nld_discrete = om.normalizer_nld.load_levels_discrete("../example_data/discrete_levels_Dy164.txt", nld0_no_nan.E)
nld_discrete.plot(kind='step')

[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.6 2.8 3.  3.2 3.4
 3.6 3.8 4.  4.2 4.4 4.6 4.8 5.  5.2 5.4 5.6 5.8 6.  6.2 6.4]
33
[0 1 2]
[3 4 5 6 7]
[ 8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
[32]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='Energy [MeV]'>)

In [ ]:
fg_norm, fg_std_norm = om.extractor.normalize(firstgen, firstgen_std)
logl = FG_loglike(firstgen, firstgen_std, nld0.E)
fg_op = LogLike2(logl)

pre_points = np.ones(len(nld0.E) - N_ρ, dtype=float)

with pm.Model() as model:
    
    # Define ρ
    #σ_ρ = om.FermiDirac("σ_ρ", mu=1.2, lam=10., shape=N_ρ)
    
    #ρ0_pre = pm.HalfFlat("ρ0_pre", shape=len(idx_pre))
    #ρ0_mid = pm.HalfFlat("ρ0_mid", shape=len(idx_mid))
    #ρ0_end = pm.HalfFlat("ρ0_end", shape=len(idx_end))
    
    #ρ_pre = pm.Normal("ρ_pre", mu=ρ0_pre, sigma=σ_ρ[idx_pre]*ρ0_pre, shape=len(idx_pre))
    #ρ_mid = pm.Normal("ρ_mid", mu=ρ0_mid, sigma=σ_ρ[idx_mid]*ρ0_mid, shape=len(idx_mid))
    #ρ_end = pm.Normal("ρ_end", mu=ρ0_end, sigma=σ_ρ[idx_end]*ρ0_end, shape=len(idx_end))
    
    #Temp = om.FermiDirac("Temp", mu=2., lam=10.)
    #E0 = pm.Normal("E0", mu=0, sigma=10.)
    
    #ρ_CT = pm.math.exp((E_nld[idx_model] - E0)/Temp)/Temp
    
    
    #ρ_discrete = pm.Normal("ρ_discrete", mu=nld_discrete.values[idx_discrete], 
    #                       sigma=nld_discrete.values[idx_discrete]*σ_ρ[idx_discrete], shape=len(idx_discrete))
    
    #ρ_model = pm.Normal("ρ_model", mu=ρ_CT, sigma=ρ_CT*σ_ρ[idx_model], shape=len(idx_model))
    
    # Next we define ρ proper
    #ρ = pm.math.concatenate((ρ_pre, ρ_discrete, ρ_mid, ρ_model, ρ_end))
    #ρ = pm.HalfFlat("ρ", shape=N_ρ)
    
    # Define transmission coefficients
    #σ_T = om.FermiDirac("σ_ρ", mu=1.2, lam=10., shape=N_T)
    #T = pm.HalfFlat("T", shape=N_T)
    
    # Models
    
    #Γ_γ_mu = pm.math.sum(T) # At the moment we will use a super hacky solution. We do not implement the integral just yet
    
    #ρ_mat = pm.math.concatenate((theano.tensor.as_tensor_variable(pre_points), ρ))
    #T_mat = pm.math.concatenate((T, theano.tensor.as_tensor_variable(np.array([0], dtype=float))))
    x = pm.HalfFlat("x", shape=len(firstgen.Eg)+len(nld0))
    #x = pm.math.concatenate((T_mat, ρ_mat))
    
    #Pth = fg_op(x)
    #Pth = ρ_mat * T_mat
    #Pth /= pm.math.sum(Pth, axis=1, keepdims=True)
    
    # We flatten and remove all that are not
    #mu_FG = pm.math.flatten(Pth)[index_keep]
    
    # Observables
    
    #Γ_γ = pm.Normal("Γ_γ", mu=Γ_γ_mu, sigma=Γ_γ_obs[1], observed=Γ_γ_obs[0])
    #ρ_Sn = pm.Normal("ρ_Sn", mu=pm.math.exp((Sn - E0)/Temp)/Temp, sigma=rhoSn[1], observed=rhoSn[0])
    #P = pm.Normal("P", mu=mu_FG, sigma=fg_std_norm.flatten()[index_keep], observed=fg_norm.flatten()[index_keep])
    P = pm.DensityDist("P", lambda x: fg_op(x), observed={'x': x})
    
    # Sample
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.
Multiprocess sampling (4 chains in 4 jobs)
Slice: [x]


In [5]:



def my_model(theta, x):
    m, c = theta
    return m*x + c

# set up our data
N = 10  # number of data points
sigma = 1.0  # standard deviation of noise
x = np.linspace(0.0, 9.0, N)

mtrue = 0.4  # true gradient
ctrue = 3.0  # true y-intercept

truemodel = my_model([mtrue, ctrue], x)

# make data
np.random.seed(716742)  # set random seed, so the data is reproducible each time
data = sigma * np.random.randn(N) + truemodel

ndraws = 3000  # number of draws from the distribution
nburn = 1000  # number of "burn-in points" (which we'll discard)

# create our Op
logl = LogLike(loglike, data, x, sigma)

# use PyMC3 to sampler from log-likelihood
with pm.Model():
    # uniform priors on m and c
    m = pm.Uniform("m", lower=-10.0, upper=10.0)
    c = pm.Uniform("c", lower=-10.0, upper=10.0)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([m, c])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist("likelihood", lambda v: logl(v), observed={"v": theta})

    trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [c]
>Slice: [m]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 16 seconds.


MissingInputError: Input 0 of the graph (indices start from 0), used to compute sigmoid(c_interval__), was not provided and not given a value. Use the Theano flag exception_verbosity='high', for more information on this error.

In [ ]:
FG_loglike

In [79]:
# Test a toy model to make sure it works...
nld_values = np.concatenate((np.mean(trace['ρ_pre'], axis=0), np.mean(trace['ρ_discrete'], axis=0),
                             np.mean(trace['ρ_mid'], axis=0), np.mean(trace['ρ_model'], axis=0),
                             np.mean(trace['ρ_end'], axis=0)))
nld_std = np.concatenate((np.std(trace['ρ_pre'], axis=0), np.std(trace['ρ_discrete'], axis=0),
                          np.std(trace['ρ_mid'], axis=0), np.std(trace['ρ_model'], axis=0),
                          np.std(trace['ρ_end'], axis=0)))

nld_normed = om.Vector(E=E_nld, values=nld_values, std=nld_std)
fig, ax = nld_normed.plot()
nld_discrete.plot(ax=ax, kind='step')
ax.errorbar(Sn, rhoSn[0], yerr=rhoSn[1], fmt="o")
ax.plot(np.linspace(3, Sn, 1001), np.exp((np.linspace(3, Sn, 1001) - np.mean(trace['E0']))/np.mean(trace['Temp']))/np.mean(trace['']))
ax.semilogy()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]